In [1]:
import numpy as np
import itertools
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter
from functools import reduce
from sklearn.metrics import accuracy_score, f1_score
stopwords = stopwords.words('english')

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip 

--2022-01-13 17:57:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K   253KB/s    in 0.8s    

2022-01-13 17:57:47 (253 KB/s) - ‘smsspamcollection.zip’ saved [203415/203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


### Spam Classifier using Naive Bayes Algorithm

In this notebook, we will train our model on Naive Bayes algorithm from scratch. 

$$
P(C \mid D)=\frac{P(D \mid C) P(C)}{P(D)}
$$
where $C$ stands for class (spam/non-spam) and $D$ stands for document.

1. $P(C)$ stands for prior probabilities
2. $P(D|C)$ stands for likelihoods
3. Since $P(D)$ is constant for different classes:

$$
P(C \mid D)\propto P(D \mid C) P(C)
$$

- As far as the proportional constant is concerned, we normalize probabilities such that they sum to 1. 
- We then compare these probabilities to assign labels. 
- Also, modelling $P(D)$ is difficult.

#### Important Points:
1. So as we learn the joint distribution $P(C,D)$, and not exactly the posterior distribution $P(C|D)$, this is a generative model. 
2. Naive Bayes assumes that the features/words are independent of each other given class i.e. features are **conditionally independent** of each other. 
3. This means that the order of words in sentence doesn't matter. It considers a sentence as **bag of words** and doesn't account for context.
4. However, it is known to work well in general which is why it's quite popular. It can have high bias due to the above reason, but then also has low variance.

#### Naive Bayes Algorithm

The steps are as follows:
1. Preprocess the data
2. Split data (stratified if needed) into training and validation sets
3. On Training data:
    1. Calculate prior probabilities 
    2. Calculate likelihoods
        1. Use Laplace Smoothing to avoid zero probabilities
4. Evaluate on validation data

In [3]:
def preprocess(sentence, tokenizer):
    words = tokenizer.tokenize(sentence)
    words = [word.lower() for word in words if word.isalnum() and word not in stopwords]
    sent = ' '.join(words)
    return sent

def get_corpus(path, tokenizer):
    with open(path) as f:
        data = f.read().splitlines()
    data = list(map(lambda x: x.split('\t'), data))
    data = np.array(data)    
    sents = np.array(list(map(lambda sent: preprocess(sent, tokenizer), data[:, 1])))
    corpus = np.dstack((data[:,0],sents)).squeeze()
    return corpus, sents

In [4]:
# splitting data into train and validation sets
def get_splits(corpus, val_pct):
    val_size = int(val_pct * len(corpus))
    idx = np.arange(len(corpus))
    np.random.shuffle(idx)

    val_idx = idx[-val_size:]
    train_idx = idx[:-val_size]
    train_corpus = corpus[train_idx]
    val_corpus = corpus[val_idx]
    return train_corpus, val_corpus

# stratified splitting data 
def get_stratified_splits(corpus, val_pct):
    corpus1 = corpus[corpus[:,0]=='ham']
    corpus2 = corpus[corpus[:,0]=='spam']
    assert len(corpus) == len(corpus1) + len(corpus2)

    train_corpus1, val_corpus1 = get_splits(corpus1, val_pct)
    train_corpus2, val_corpus2 = get_splits(corpus2, val_pct)
    train_corpus = np.vstack((train_corpus1, train_corpus2))
    val_corpus = np.vstack((val_corpus1, val_corpus2))
    return train_corpus, val_corpus    

In [5]:
tokenizer = RegexpTokenizer(r'\w+')    
val_pct = 0.2
path = 'SMSSpamCollection'
corpus, sents = get_corpus(path, tokenizer)
train_corpus, val_corpus = get_stratified_splits(corpus, val_pct)

print('Number of Training examples:', len(train_corpus))
print('Number of Validation examples:', len(val_corpus),'\n')
print('Checking class imablance..')
print('Percentage of non-spam messages:', np.mean(corpus[:,0]=='ham').round(2),'\n')

words = list(map(lambda sent: sent.split(' '), sents))
vocab = list(set(itertools.chain(*words)))
print('Vocabulary size:', len(vocab))

Number of Training examples: 4460
Number of Validation examples: 1114 

Checking class imablance..
Percentage of non-spam messages: 0.87 

Vocabulary size: 8729


In [6]:
# get likelihoods
def get_cond_prob(corpus):
    words = list(map(lambda x: x.split(' '), corpus[:,1]))
    freq_dict = dict(Counter(list(itertools.chain(*words))))
    extra_tokens = set(vocab) - set(freq_dict.keys())
    extra_freq_dict = dict(zip(extra_tokens, [0] * len(extra_tokens)))
    freq_dict.update(extra_freq_dict)

    # laplace smoothing 
    # adding 1 for solving issues of zero probabilities
    cond_prob = {k:(v+1) / (sum(freq_dict.values()) + len(freq_dict)) for k,v in freq_dict.items()}
    return cond_prob

# get posterior probabilities
def get_probs(words):
    probs = []
    for label in labels:
        prior = class_priors[label]
        cond_probs = list(map(lambda x: likelihoods[label][x], words))
        likelihood = reduce(lambda x,y: x*y, cond_probs)
        prob = prior * likelihood
        probs.append(prob)
    
    # normalizing probabilities
    probs = probs / np.sum(probs)
    return probs    

In [7]:
# get prior probabilities
labels = list(set(corpus[:,0]))
class_priors = {}
for label in labels:
    class_priors[label] = sum(train_corpus[:,0]==label) / len(train_corpus)

# get likelihoods
likelihoods = {}
for label in labels:
    label_train_corpus = train_corpus[train_corpus[:,0]==label]
    likelihoods[label] = get_cond_prob(corpus)

In [8]:
def get_predictions(val_corpus):
    sents = list(map(lambda x: x.split(' '), val_corpus[:,1]))
    y_pred = []
    for words in sents:
        probs = get_probs(words)
        label_idx = probs.tolist().index(max(probs))
        pred = labels[label_idx]
        y_pred.append(pred)
    y_pred = np.array(y_pred)
    return y_pred

# predictions
y_pred = get_predictions(val_corpus).tolist()

# ground truth
y_val = val_corpus[:,0].tolist()  
assert len(y_val) == len(y_pred)    

# metrics
accuracy_score(y_val, y_pred).round(2), f1_score(y_val, y_pred, pos_label='ham').round(2)

(0.87, 0.93)